In [31]:
from matminer.datasets import load_dataset

# dielectric = load_dataset("matbench_dielectric")
# log_gvrh = load_dataset("matbench_log_gvrh")
# jdft2d = load_dataset("matbench_jdft2d")

boltztrap_mp = load_dataset("boltztrap_mp")

# matbench_dielectric
matbench_expt_gap
# matbench_expt_is_metal
# matbench_glass
# matbench_jdft2d
# matbench_steels

Fetching boltztrap_mp.json.gz from https://ndownloader.figshare.com/files/13297511 to /Users/muon/opt/anaconda3/lib/python3.9/site-packages/matminer/datasets/boltztrap_mp.json.gz


Fetching https://ndownloader.figshare.com/files/13297511 in MB: 14.020608MB [00:00, 363.05MB/s]                         


In [41]:
import pandas as pd
# dielectric.iloc[0].structure 
# boltztrap_mp.columns

# expt_gap.composition

pd.Series(list(set(boltztrap_mp.formula) & set(expt_is_metal.composition)))


0        SrCrO4
1          PrF3
2           SnS
3      Tl9SbSe6
4       K4GeSe4
         ...   
362         TlS
363      CrPbO4
364    LiNiP2O7
365         LiF
366          KF
Length: 367, dtype: object

In [49]:
boltztrap_mp.head()
boltztrap_mp.columns

Index(['mpid', 'pf_n', 'pf_p', 's_n', 's_p', 'formula', 'm_n', 'm_p',
       'structure'],
      dtype='object')

In [53]:
import torch
from torch_geometric.data import Data
import numpy as np
import pandas as pd
from tqdm import tqdm

periodic_table = pd.read_csv('data/periodic_table.csv')

# Find the largest element in the dataset
# Returns the atomic number of the largest element
def get_largest_element(df):
    largest_element = 0
    for idx, entry in df.iterrows():
        struct = entry.structure
        for site in struct._sites:
            symbol = str(list(site._species._data.keys())[0])
            atomic_number = periodic_table.AtomicNumber[periodic_table['Symbol'] == symbol].values[-1]
            if atomic_number > largest_element:
                largest_element = atomic_number
    return largest_element

# Create a fully connected graph from a pymatgen structure
# Returns an array of shape (2, n_edges) where each column is an edge
# This is the format required by the PyTorch Geometric library
def make_edge_indices(entry):
    n_nodes = len(entry.structure._sites)
    edge_index = []
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j:
                edge_index.append([i, j])
    return torch.tensor(edge_index).transpose(0, 1)

# Create a graph from a Pandas DataFrame of pymatgen structures
# Returns a tuple of (feature_matrix, coord_matrix, label)
# feature_matrix is a matrix of shape (n_nodes, n_features)
# coord_matrix is a matrix of shape (n_nodes, 3)
# label is a scalar
def get_features_and_coords(df):
    largest_element = get_largest_element(df)
    data = []

    for idx, entry in tqdm(df.iterrows(), desc="Building material graphs"):
        struct = entry.structure

        feature_matrix = []
        coord_matrix = []

        # Features
        for site in struct._sites:
            feature_vec = [0 for _ in range(largest_element)] # create a vector of zeros
            symbol = str(list(site._species._data.keys())[0])
            atomic_number = periodic_table.AtomicNumber[periodic_table['Symbol'] == symbol].values[-1]
            feature_vec[atomic_number - 1] = 1 # one-hot encode atomic number
            feature_matrix.append(feature_vec)

        # Coordinates
        for site in struct._sites:
            coords = site._frac_coords
            coord_matrix.append(coords)

        coord_matrix = torch.FloatTensor(np.array(coord_matrix))
        feature_matrix = torch.FloatTensor(np.array(feature_matrix))

        # Labels
        labels = {}
        for col in df.columns:
            if col != 'structure':
                labels[col] = torch.tensor(entry[col])

        if (feature_matrix is not None) and (len(feature_matrix) > 1): 
            edge_index=make_edge_indices(entry)
            if len(edge_index.shape) > 1:
                datum = Data(x=feature_matrix, edge_index=edge_index, y=labels, pos=coord_matrix)
                data.append(datum)

    return data

In [56]:
from tqdm import tqdm
df = boltztrap_mp.drop(columns=['mpid', 'formula'])
dataset = get_features_and_coords(df)


In [ ]:
dataset[0].y